# show the mechanism of try/Except/Finally/Return Handling

# env 

```bash

date
    Thu Apr 10 00:00:00 PM CEST 2025

cat /etc/os-release 
    PRETTY_NAME="Debian GNU/Linux 12 (bookworm)"
    NAME="Debian GNU/Linux"
    VERSION_ID="12"
    VERSION="12 (bookworm)"
    VERSION_CODENAME=bookworm
    ID=debian

python3 --version
    Python 3.11.2 
```

In [ ]:
## simple block from {here)[https://docs.python.org/3.11/tutorial/errors.html]

In [4]:
def bool_return():
    try:
        return True
    finally:
        return False

bool_return()

False

# Handling ZeroDivisionError

In [2]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)


Handling run-time error: division by zero


# Raising Exceptions

In [5]:
try:
    # trigger an error
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    # raise

An exception flew by!


# with raise

In [6]:
try:
    # trigger an error
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise

An exception flew by!


NameError: HiThere

# Exception Chaining

In [9]:
try:
    open("database.sqlite")
except OSError:
    print(RuntimeError("unable to handle error"))

unable to handle error


# with raise in except leg instead print

In [10]:
try:
    open("database.sqlite")
except OSError:
    raise RuntimeError("unable to handle error")

RuntimeError: unable to handle error

# raise RuntimeError from exc -  exc must be exception instance or None

- To indicate that an exception is a direct consequence of another, the raise statement allows an optional from clause:

```python
raise RuntimeError from exc
```

In [16]:
def func():
    raise ConnectionError
try:
    func()
except ConnectionError as exc:
    raise RuntimeError('Failed to open database') from exc

RuntimeError: Failed to open database

# It also allows disabling automatic exception chaining using the from None idiom
- Don't see any error message

In [17]:
try:
    open('database.sqlite')
except OSError:
    raise RuntimeError from None

RuntimeError: 

# User-defined Exceptions

# Defining Clean-up Actions without exception block

<div class="alert alert-block alert-info">
- If a finally clause is present, the finally clause will execute as the last task before 
- the try statement completes. 
- The finally clause runs whether or not the try statement produces an exception. 
</div>

In [21]:
try:
    raise KeyboardInterrupt
# except: block should missing here  
finally:
    print('Goodbye, world!')
    

Goodbye, world!


KeyboardInterrupt: 

# A more complicated example:

In [22]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

# good case finally block will execute

In [23]:
divide(2, 1)

result is 2.0
executing finally clause


# bad case finally block will execute despite it

In [24]:
divide(2, 0)

division by zero!
executing finally clause


save
# unhandled exception caught/catch => TypeError: unsupported operand type(s) for /: 'str' and 'str' finally block will execute


In [25]:
divide("2", "1")

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

# ExceptionGroup - Raising and Handling Multiple Unrelated Exceptions

<div class="alert alert-block alert-info">
📝 There are situations where it is necessary to report several exceptions that have occurred. This is often the case in concurrency frameworks, when several tasks may have failed in parallel, but there are also other use cases where it is desirable to continue execution and collect multiple errors rather than raise the first exception.

📝 The builtin ExceptionGroup wraps a list of exception instances so that they can be raised together. It is an exception itself, so it can be caught like any other exception.
</div>

In [26]:
def f():
    raise ExceptionGroup(
        "group1",
        [
            OSError(1),
            SystemError(2),
            ExceptionGroup(
                "group2",
                [
                    OSError(3),
                    RecursionError(4)
                ]
            )
        ]
    )

try:
    f()
except* OSError as e:
    print("There were OSErrors")
except* SystemError as e:
    print("There were SystemErrors")

There were OSErrors
There were SystemErrors


  + Exception Group Traceback (most recent call last):
  |   File "/home/trapapa/ib_async_python_try_out/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3667, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_8611/1308333313.py", line 18, in <module>
  |     f()
  |   File "/tmp/ipykernel_8611/1308333313.py", line 2, in f
  |     raise ExceptionGroup(
  | ExceptionGroup: group1 (1 sub-exception)
  +-+---------------- 1 ----------------
    | ExceptionGroup: group2 (1 sub-exception)
    +-+---------------- 1 ----------------
      | RecursionError: 4
      +------------------------------------


# Enriching Exceptions with Notes

In [28]:
def f():
    raise OSError('operation failed')

excs = []
for i in range(3):
    try:
        f()
    except Exception as e:
        e.add_note(f'Happened in Iteration {i+1}')
        excs.append(e)

raise ExceptionGroup('We have some problems', excs)
  

  + Exception Group Traceback (most recent call last):
  |   File "/home/trapapa/ib_async_python_try_out/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3667, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_8611/980257444.py", line 12, in <module>
  |     raise ExceptionGroup('We have some problems', excs)
  | ExceptionGroup: We have some problems (3 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_8611/980257444.py", line 7, in <module>
    |     f()
    |   File "/tmp/ipykernel_8611/980257444.py", line 2, in f
    |     raise OSError('operation failed')
    | OSError: operation failed
    | Happened in Iteration 1
    +---------------- 2 ----------------
    | Traceback (most recent call last):
    |   File "/tmp/ipykernel_8611/980257444.py", line 7, in <module>
    |     f()
    |   File "/tmp/ipykernel_8611/980257444.py", line 2, 